# Learning on League of Legends Data

### Written by Brian Perrett for funsies

In [2]:
from __future__ import division
import numpy as np
import csv
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import ast

In [3]:
with open("F:/Box Sync/Documents/Python Documents/riot/champids.txt", "r") as f:
    champion_dict = ast.literal_eval(f.read())
print(champion_dict)
global champion_dict

{'133': 'Quinn', '131': 'Diana', '134': 'Syndra', '24': 'Jax', '25': 'Morgana', '26': 'Zilean', '27': 'Singed', '20': 'Nunu', '21': 'Miss Fortune', '22': 'Ashe', '23': 'Tryndamere', '28': 'Evelynn', '29': 'Twitch', '161': "Vel'Koz", '4': 'Twisted Fate', '8': 'Vladimir', '120': 'Hecarim', '121': "Kha'Zix", '122': 'Darius', '267': 'Nami', '266': 'Aatrox', '126': 'Jayce', '127': 'Lissandra', '268': 'Azir', '59': 'Jarvan IV', '58': 'Renekton', '55': 'Katarina', '54': 'Malphite', '57': 'Maokai', '56': 'Nocturne', '51': 'Caitlyn', '50': 'Swain', '53': 'Blitzcrank', '412': 'Thresh', '115': 'Ziggs', '114': 'Fiora', '117': 'Lulu', '89': 'Leona', '111': 'Nautilus', '110': 'Varus', '113': 'Sejuani', '112': 'Viktor', '82': 'Mordekaiser', '83': 'Yorick', '80': 'Pantheon', '81': 'Ezreal', '119': 'Draven', '84': 'Akali', '85': 'Kennen', '429': 'Kalista', '3': 'Galio', '7': 'LeBlanc', '421': "Rek'Sai", '420': 'Illaoi', '245': 'Ekko', '102': 'Shyvana', '103': 'Ahri', '101': 'Xerath', '106': 'Volibear',

In [4]:
def getDataNoInGame(filename):
    features = []
    x = []
    y = []
    with open(filename, "r") as f:
        reader = csv.reader(f)
        r = 0
        for row in reader:
            if r == 0:
                features = row[1:-14]
                r += 1
                continue
            if len(row) == 0:
                pass
            else:
                x.append(list(map(int, row[1:-14])))
                y.append(list(map(int, row[-2]))[0])
#         features = x[0]
    return np.array(features), np.array(x), np.array(y)

In [5]:
def getData(filename):
    """
    Includes data like first baron and first inhibitor.
    """
    features = []
    x = []
    y = []
    with open(filename, "r") as f:
        reader = csv.reader(f)
        r = 0
        for row in reader:
            if r == 0:
                features = row[1:-2]
                r += 1
                continue
            if len(row) == 0:
                pass
            else:
                x.append(list(map(int, row[1:-2])))
                y.append(list(map(int, row[-2]))[0])
#         features = x[0]
    return np.array(features), np.array(x), np.array(y)

In [6]:
def testCorrect(trainsize, x, y):
    correct = 0
    for i, x1 in enumerate(x[trainsize:]):
        ans = clf.predict([x1])
    #     print(ans[0], int(y[trainsize + i]))
        if int(ans[0]) == int(y[trainsize + i]):
            correct += 1
    print("{} of {}".format(correct, len(x[trainsize:])))
    print(correct/len(x[trainsize:]))

In [7]:
def getTopFeatures(clf, n):
    indeces = clf.coef_[0].argsort()[-10:][::-1]
    for ind in indeces:
        if "champ" in features[ind]:
            champ = features[ind].split("champ")
            f = "{}{}".format(champ[0], champion_dict[champ[1]])
        else:
            f = features[ind]
        print(clf.coef_[0][ind], f)

In [8]:
def getWorstFeatures(clf, n):
    indeces = clf.coef_[0].argsort()[:10]
    for ind in indeces:
        if "champ" in features[ind]:
            champ = features[ind].split("champ")
            f = "{}{}".format(champ[0], champion_dict[champ[1]])
        else:
            f = features[ind]
        print(clf.coef_[0][ind], f)


### Diamond

# Stochastic Gradient Descent

In [9]:
features, x, y = getData("diamond.csv")
trainsize = int(.6 * len(x))

In [10]:
len(x)

4050

In [11]:
clf = SGDClassifier(loss="log", penalty="l2", n_iter=50)
clf.fit(x[:trainsize], y[:trainsize])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [12]:
testCorrect(trainsize, x, y)

1411 of 1620
0.870987654321


<h4>Reading the top features list</h4>
<ul>
    <li>t100 = Team 1</li>
    <li>t200 = Team 2</li>
    <li>Basically, if you see a t200 on the topfeatures list, it means you want the other team to have that feature</li>
</ul>

In [13]:
getTopFeatures(clf, 10)

(4.7953455844123329, 't100firstinhibitor')
(2.8398065867578812, 't200:p10:Tryndamere')
(2.7132403942563212, 't100:p1:Akali')
(2.6592542376319486, 't200:p8:Shaco')
(2.6454619154157348, "t200:p10:Vel'Koz")
(2.5592554082180508, 't100:p2:Vayne')
(2.5542158265325039, 't200:p6:Braum')
(2.4720206064685226, 't200:p10:Veigar')
(2.4410791155160378, 't100firstbaron')
(2.4019936685062784, 't100:p2:Fizz')


The SGD algorithm put the largest weight on who took first tower, which makes a lot of sense, but I'm surprised it isn't something else, like first baron, or inhibitor.

Wow okay it totally is first inhibitor, I had an off by one error in my indexing.  Everything makes sense now.

# Support Vector Machine

In [14]:
clf = SVC(kernel="linear")
clf.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
testCorrect(trainsize, x, y)

1582 of 1620
0.976543209877


The SVM was way slower, but with almost perfect predictions

In [16]:
getTopFeatures(clf, 10)

(1.9221614242489031, 't100firstinhibitor')
(1.4134330937206883, 't200:p8:Shaco')
(1.3393736042535727, 't200:p6:Xerath')
(1.2869091698333024, 't100:p4:Nami')
(1.2636548312007148, 't100:p2:Sejuani')
(1.2519879847705975, 't100:p1:Akali')
(1.2457964324805502, 't100:p2:Vayne')
(1.2235508451634636, 't200:p9:Karthus')
(1.1686349919970627, 't100:p1:Taric')
(1.1549587773247338, 't200:p8:Dr. Mundo')


Basically, it looks like if the other team has a shaco on their team, it's a good thing for you.  Both algorithms agreed there.

### Bronze

# SGD

In [17]:
features, x, y = getData("bronze.csv")
trainsize = int(.6 * len(x))

In [18]:
len(x)

2394

In [19]:
clf = SGDClassifier(loss="log", penalty="l2", n_iter=50)
clf.fit(x[:trainsize], y[:trainsize])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [20]:
testCorrect(trainsize, x, y)

811 of 958
0.846555323591


In [21]:
getTopFeatures(clf, 10)

(4.0780341544599672, 't100firstinhibitor')
(2.5687310089850075, 't100:p3:Lux')
(2.5534177132363873, 't200:p6:Teemo')
(2.5102583295827352, 't100:p4:Yasuo')
(2.4331208700436329, 't100:p3:Jax')
(2.4205954521501969, 't100:p3:Talon')
(2.4112080514270793, 't200:p10:Hecarim')
(2.374823489169323, 't200:p6:Annie')
(2.2954659333231957, 't200:p7:Tryndamere')
(2.2550787553853411, 't200:p9:Blitzcrank')


# SVM

In [22]:
clf = SVC(kernel="linear")
clf.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
testCorrect(trainsize, x, y)

944 of 958
0.985386221294


In [24]:
getTopFeatures(clf, 10)

(1.5563313625313706, 't200:p10:Hecarim')
(1.3725625528065544, 't100firstinhibitor')
(1.3466930431097506, 't100:p4:Nasus')
(1.2287842619682321, 't100:p5:Draven')
(1.2132124630750809, 't200:p10:Nidalee')
(1.2053695635800641, 't200:p7:Volibear')
(1.1892381567730586, 't200:p6:Talon')
(1.161076445045945, 't200:p6:Gnar')
(1.1101318204266222, 't100:p5:Sion')
(1.0891531280511229, 't100:p1:Annie')


In [25]:
getWorstFeatures(clf, 10)

(-1.6203523849847805, 't200:p10:Illaoi')
(-1.4782681058126228, 't200firstinhibitor')
(-1.3142544008570698, "t200:p6:Kog'Maw")
(-1.2448348290728046, 't200:p7:Azir')
(-1.2402458430467633, 't200:p9:Twisted Fate')
(-1.1827097915564944, 't200:p8:Karthus')
(-1.1575872702935666, 't100:p1:Zed')
(-1.1551235540340952, 't200:p8:Sejuani')
(-1.0832806158864661, 't100:p1:Nunu')
(-1.0397595238540016, 't100:p1:Xerath')


Apparently hecarim is not good on this patch or something.

### Challenger

In [26]:
features, x, y = getData("challenger.csv")
trainsize = int(.8 * len(x))

In [27]:
clf = SGDClassifier(loss="log", penalty="l2", n_iter=50)
clf.fit(x[:trainsize], y[:trainsize])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [28]:
testCorrect(trainsize, x, y)

62 of 71
0.87323943662


In [29]:
getTopFeatures(clf, 10)

(9.2586255948382767, 't100firstinhibitor')
(3.7565552243529998, 't100firstbaron')
(3.3940741582075091, 't200:p6:Lucian')
(3.3369911715648146, 't200:p9:Riven')
(3.3343600665079878, 't100:p4:Wukong')
(3.3325219357224443, 't100:p5:Vayne')
(3.1399380092940716, 't200:p10:spell112')
(2.9670288920162071, 't100:p4:Janna')
(2.7963696663082223, 't200:p8:Ezreal')
(2.7907344689232145, 't200:p10:Corki')


In [30]:
getWorstFeatures(clf, 10)

(-10.689015417418142, 't200firstinhibitor')
(-6.1745348150526489, 't200firstbaron')
(-3.7201689568111775, 't200:p10:spell114')
(-3.3219388866008033, 't100:p3:Gragas')
(-2.729922620612883, 't100:p2:spell13')
(-2.6749337303808884, 't100:p1:Viktor')
(-2.6670562367840533, "t100:p1:Rek'Sai")
(-2.5987716570977666, 't100:p4:Bard')
(-2.4199281798587533, 't200:p7:spell17')
(-2.294666173835473, 't100:p3:spell112')


This may just be that my challenger dataset is relatively smaller than the others, but it is interesting to see the large weight on inhibitor and baron.  

It is much larger than it was in Bronze which would imply that getting first baron or inhib for challenger games is much more indicitive of winning.

# SVM

In [31]:
clf = SVC(kernel="linear")
clf.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [32]:
testCorrect(trainsize, x, y)

71 of 71
1.0


In [33]:
getTopFeatures(clf, 10)

(0.84376448830477058, 't100firstinhibitor')
(0.39022169412339591, 't200:p10:Corki')
(0.35319143837353273, 't200:p9:Lee Sin')
(0.34980695636668141, 't100:p4:Wukong')
(0.34157096431957373, 't200:p10:Ekko')
(0.33817091259529097, 't200:p7:spell11')
(0.33729562183535411, 't100firstbaron')
(0.30714224877813301, 't200:p7:Ahri')
(0.2891027284345008, 't200:p7:spell121')
(0.28865677196628964, 't100firstdragon')


# What if we didn't know about barons and inhibs?

### Diamond

#### SGD

In [34]:
features, x, y = getDataNoInGame("diamond.csv")
trainsize = int(.6 * len(x))

In [35]:
clf = SGDClassifier(loss="log", penalty="l2", n_iter=50)
clf.fit(x[:trainsize], y[:trainsize])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [36]:
testCorrect(trainsize, x, y)

844 of 1620
0.520987654321


SGD has a much harder time discerning who wins, which means that the champions are more or less balanced.
Only classified 52 % of them correctly.  Basically a coin flip.

#### SVM

In [37]:
clf = SVC(kernel="linear")
clf.fit(x, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
testCorrect(trainsize, x, y)

1270 of 1620
0.783950617284


Wow almost 80% correct on the test set??  Imbalance?

In [39]:
getTopFeatures(clf, 10)

(1.8183912726840334, 't200:p9:Rumble')
(1.6730483627122275, 't200:p6:Yorick')
(1.6678794419393672, 't100:p1:Akali')
(1.6447259451347747, 't100:p1:Taric')
(1.6005601262474496, 't200:p6:Teemo')
(1.5721088539867876, 't200:p9:Zac')
(1.4964099580946537, 't200:p8:Twitch')
(1.4839422463911134, 't200:p9:Heimerdinger')
(1.4432712419660707, 't200:p8:Sejuani')
(1.4190523825849422, 't100:p3:Illaoi')


Suggests that Rumble and Yorick are not good picks right now.  Whereas Akali is.